<a href="https://colab.research.google.com/github/lautitorraca/7506R-2C2023-GRUPO24/blob/main/7506R_TP1_GRUPO24_CHP1_ENTREGA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabajo Practico 1: Reservas de Hotel

## 𝔾𝕣𝕦𝕡𝕠 𝟚𝟜 (Merequetengue👍)

      ➢ Torraca Lautaro - 108813
      ➢ Negrotti Gianluca - 108184
      ➢ Marco Tosi - 107237

In [1]:
# Imports for notebook

import pandas as pd
from IPython.display import display
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import seaborn as sns


url = 'TP1/DataSets/hotels_test.csv'

plt.style.use('seaborn-v0_8-pastel')

============================== TODO ==============================


-> Ver donde ponemos la Fecha y el ID.

-> Leer el Paper y ver si hay variables irrelevantes.
(Yo creo que sin contar de company el resto si puede llegar a ser util para algo)


-> Que paso con las variables con muchos Nan?

**-> Vivirla en el informe:**

# » Lectura del archivo

In [2]:
ds_original = pd.read_csv(url)
ds_base = ds_original.copy()

ParserError: ignored

# » Muestra del DataSet


In [ ]:
ds_base.head()

# » Exploración Inicial y filtrado de datos.

Analizando el DataSet, podemos observar que la columna 'company' se encuentra en un gran porcentaje vacía, por lo que se decidió eliminar la columna del DataSet con el fin de dejar lo más limpio posible el archivo y trabajar únicamente con datos que puedan ser relevantes en el trabajo práctico.

In [ ]:
ds_filtrado_company = ds_base['company']

numero_de_NaN_company = ds_filtrado_company.isna().sum()
numero_de_lineas_company = ds_filtrado_company.shape[0]

porcentaje_de_NaN_company = (numero_de_NaN_company/numero_de_lineas_company ) * 100
porcentaje_de_NaN_redondeado_company = round(porcentaje_de_NaN_company, 2)

valores_company = {'numero_de_NaN': [numero_de_NaN_company],
           'numero_de_lineas': [numero_de_lineas_company],
           'porcentaje_de_NaN': [porcentaje_de_NaN_redondeado_company]}


df_company = pd.DataFrame.from_dict(valores_company)
df_company.head()



In [ ]:
ds_filtrado_agent = ds_base['agent']

numero_de_NaN_agent = ds_filtrado_agent.isna().sum()
numero_de_lineas_agent = ds_filtrado_agent.shape[0]

porcentaje_de_NaN_agent = (numero_de_NaN_agent/numero_de_lineas_agent ) * 100
porcentaje_de_NaN_redondeado_agent = round(porcentaje_de_NaN_agent, 2)

valores_agent = {'numero_de_NaN': [numero_de_NaN_agent],
           'numero_de_lineas': [numero_de_lineas_agent],
           'porcentaje_de_NaN': [porcentaje_de_NaN_redondeado_agent]}


df_agent = pd.DataFrame.from_dict(valores_agent)
df_agent.head()

### » Tipos de las varibales

In [ ]:
ds_base.dtypes

### » Separación de variables cuantitativas del DataSet para analizar la media, mediana y moda.

In [ ]:
nombre_variables_cuantitativas = ['lead_time','arrival_date_week_number', 'stays_in_weekend_nights', 'adults', 'children','babies',
                                                         'is_repeated_guest', 'previous_cancellations', 'previous_bookings_not_canceled', 'booking_changes',
                                                         'agent', 'days_in_waiting_list', 'adr', 'required_car_parking_spaces', 'total_of_special_requests' ]

ds_cuantitativo = ds_base.loc[:, nombre_variables_cuantitativas]
df = pd.DataFrame()
df['media'] = round(ds_cuantitativo.mean(), 2)
df['mediana'] = ds_cuantitativo.median()
df['moda'] = ds_cuantitativo.mode().iloc[0]
df

In [ ]:
round(ds_cuantitativo.describe(), 2).transpose()

### » Separación de las variables cualitativas y análisis.

In [ ]:
nombre_variables_cualitativas = ['hotel', 'arrival_date_month', 'meal', 'country', 'market_segment', 'reserved_room_type', 'distribution_channel', 'assigned_room_type', 'deposit_type',
                                 'customer_type']

ds_cualitativo = ds_base.loc[:, nombre_variables_cualitativas]


### » Cantidad de valores posibles dentro del DataSet con variables cualitativas unicamente:


In [ ]:
ds_cualitativo.nunique().sort_values(ascending=True)


### » Frecuencia de los valores
Los numeros representan el porcentaje de aparacion de esa variable.

- Hoteles.

In [ ]:
round(ds_cualitativo.hotel.value_counts(normalize=True) * 100, 2)


- **Tipo de depósito.**

In [ ]:
round(ds_cualitativo.deposit_type.value_counts(normalize=True) * 100, 2)


- **Canal de distribución.**

In [ ]:
round(ds_cualitativo.distribution_channel.value_counts(normalize=True) * 100, 2)

- **Tipo de cliente.**


In [ ]:
round(ds_cualitativo.customer_type.value_counts(normalize=True) * 100, 2)

- **Comida.**

In [ ]:
round(ds_cualitativo.meal.value_counts(normalize=True) * 100, 2)

- **Segmento de mercado (método de reserva).**

In [ ]:
round(ds_cualitativo.market_segment.value_counts(normalize=True) * 100, 2)

- **Mes de llegada**

In [ ]:
round(ds_cualitativo.arrival_date_month.value_counts(normalize=True) * 100, 2)

- **País**

In [ ]:
ds_cualitativo.country.value_counts(normalize=True) * 100

## » Varibles ordinales
Los numeros representan el porcentaje de aparacion de esa variable.

In [ ]:
nombre_variables_ordinales = ['reserved_room_type', 'assigned_room_type']

ds_ordinales = ds_base.loc[:, nombre_variables_ordinales]

- **Tipo de habitación reservada.**

In [ ]:
round(ds_ordinales.reserved_room_type.value_counts(normalize=True) * 100, 2)

- **Tipo de habitación asignada**

In [ ]:
round(ds_ordinales.assigned_room_type.value_counts(normalize=True) * 100, 2)

In [ ]:
## QUEDA COMPLETAR EL GRAFICO CON %
plt.subplot(2,1,1)

a = plt.bar(ds_ordinales['reserved_room_type'].value_counts().index, ds_ordinales['reserved_room_type'].value_counts())
plt.ylabel('Porcentaje')


plt.subplot(2,1,2)

b= plt.bar(ds_ordinales['assigned_room_type'].value_counts().index, ds_ordinales['assigned_room_type'].value_counts())
plt.ylabel('Porcentaje')


# » Análisis gráfico de las distribuciones de las variables.

-  **Meses**


En el siguiente gráfico podemos observar cuales son los meses con mayor actividad de reserva en la totalidad de los hoteles.

In [ ]:

grafico_distribucion_meses = ds_cualitativo['arrival_date_month'].value_counts().plot.barh(alpha=1, title="Cantidad de reservas por mes ",xlabel='Cantidad de reservas',ylabel='Meses')

valores_barras = round(ds_cualitativo['arrival_date_month'].value_counts(normalize = True) * 100, 2)


for index, value in enumerate(valores_barras):
    plt.text(value, index, str(value) +"%")


- **Hoteles**

En el grafico a continuacion se logra observar el porcentaje en que se realizan reservas en cada hotel.

In [ ]:

hoteles = ds_cualitativo.hotel.value_counts()

df = pd.DataFrame(hoteles)

valores = df['hotel']
categoria = df.index

plt.figure(figsize=(6, 6))

plt.pie(valores, autopct=lambda p: '{:.2f}% ({:.0f})'.format(p,(p/100)*valores.sum()), startangle=0)
plt.title('Frecuencia de reservas por hotel')

plt.legend(labels=categoria, loc="best")


plt.show()

- **Tipo de deposito**

In [ ]:
tipo_deposito = ds_cualitativo.deposit_type.value_counts()

df = pd.DataFrame(tipo_deposito)

valores = df['deposit_type']
categoria = df.index

plt.figure(figsize=(6, 6))
plt.pie(valores, explode=(0.2, 0.2, 0.1) ,autopct=lambda p: '{:.2f}% ({:.0f})'.format(p,(p/100)*valores.sum()), startangle=0)

plt.title('Frecuencia de reservas por hotel')

plt.legend(labels=categoria,loc="upper right")


plt.show()



- **Canal de distribucion**

In [ ]:
tipos_de_canales = ds_cualitativo.distribution_channel.value_counts()

df = pd.DataFrame(tipos_de_canales)

valores = df['distribution_channel']
categoria = df.index

plt.figure(figsize=(6, 6))
plt.pie(valores, explode=(0, 0.05, 0.1, 0.4) ,autopct=lambda p: '{:.2f}% ({:.0f})'.format(p,(p/100)*valores.sum()), startangle=0)

plt.title('Impacto de los canales de distribucion')

plt.legend(labels=categoria,loc="upper right")

plt.show()



- **Tipo de cliente**

In [ ]:
tipos_de_clientes = ds_cualitativo.customer_type.value_counts()

df = pd.DataFrame(tipos_de_clientes)

valores = df['customer_type']
categoria = df.index

plt.figure(figsize=(6, 6))
plt.pie(valores, explode=(0, 0.05, 0.1, 0.4) ,autopct=lambda p: '{:.2f}% ({:.0f})'.format(p,(p/100)*valores.sum()), startangle=0)

plt.title('Tipos de clientes')

plt.legend(labels=categoria,loc="upper right")


plt.show()


- **Comida**

In [ ]:
tipos_de_comidas = ds_cualitativo.meal.value_counts()

df = pd.DataFrame(tipos_de_comidas)

valores = df['meal']
categoria = df.index

plt.figure(figsize=(6, 6))
plt.pie(valores, explode=(0, 0.1, 0.1, 0.1, 0.7),autopct=lambda p: '{:.2f}% ({:.0f})'.format(p,(p/100)*valores.sum()), startangle=0)

plt.title('Tipos de clientes')

plt.legend(labels=categoria,loc="upper right")


plt.show()


- **Segmento de mercado**

In [ ]:

grafico_distribucion_meses = ds_cualitativo['market_segment'].value_counts().plot.barh(alpha=1, title="Cantidad de reservas por mes",xlabel='Cantidad de reservas',ylabel='Meses')
valores_barras = round(ds_cualitativo['market_segment'].value_counts(normalize = True) * 100, 2)


for index, value in enumerate(valores_barras):
    plt.text(value, index, str(value) +"%")


- **Tipos de clienteS**



In [ ]:
tipos_de_clientes = ds_cualitativo.customer_type.value_counts()

df = pd.DataFrame(tipos_de_clientes)

valores = df['customer_type']
categoria = df.index

plt.figure(figsize=(6, 6))
plt.pie(valores, explode=(0, 0.05, 0.1, 0.4) ,autopct=lambda p: '{:.2f}% ({:.0f})'.format(p,(p/100)*valores.sum()), startangle=0)

plt.title('Tipos de clientes')

plt.legend(labels=categoria,loc="upper right")


plt.show()

- **Paises con mas reservas en los hoteles**

In [ ]:

#top_paises = ds_cualitativo['country'].value_counts()
sns.histplot(data=ds_cualitativo, x=ds_cualitativo.babies, bins=5)#, title="Top países con más reservas", xlabel='Países', ylabel='Cantidad de reservas')


In [ ]:
top_paises = ds_cualitativo['country'].value_counts().head(31)
paises = top_paises.plot.bar(alpha=1, title="Top países con más reservas", xlabel='Países', ylabel='Cantidad de reservas')


# » Grafico de los valores atipicos

- **Cantidad de adultos por habitacion**

In [ ]:

grf = ds_base.adults.plot(kind = 'hist', bins= 60, log=True, ylabel='Cantidad')


In [ ]:
grf = sns.boxplot(y=ds_base.adults, data= ds_base ).set(title="Boxplot cantidad de adultos por habitación",xlabel="",ylabel="Cantidad")

- **Estadia los fines de semana en el hotel**

In [ ]:
grf = sns.boxplot(y=ds_base.stays_in_weekend_nights, data= ds_base).set(title="Boxplot cantidad de fines de semana en el hotel",xlabel="",ylabel="Cantidad")

- **Cantidad de bebes en cada familia**

In [ ]:
grf = sns.boxplot(y=ds_base.babies, data= ds_base).set(title="Boxplot cantidad de bebes en cada familia",xlabel="",ylabel="Cantidad")

- **Cancelaciones previas de la habitacion**

In [ ]:
grf = sns.boxplot(y=ds_base.previous_cancellations, data= ds_base).set(title="Boxplot cantidad cancelaciones previas",xlabel="",ylabel="Cantidad")

- **Dias en la lista de espera**

In [ ]:
grf = sns.boxplot(y = ds_base.days_in_waiting_list, data = ds_base).set(title="Boxplot cantidad dias de espera en la lista",xlabel="",ylabel="Cantidad")

- **ADR**

In [ ]:
grf = sns.boxplot(y= ds_base.adr, data= ds_base ).set(title="Boxplot de ADR ",xlabel="",ylabel="Cantidad")

# » Análisis de las correlaciones entre las variables.


In [ ]:
ds_base.corr(numeric_only = True)

In [ ]:
plt.figure(figsize = (20,10))
sns.heatmap(ds_base.corr(numeric_only = True),annot = True, cmap= "Pastel1")

# » Analisis de nulos en el DataSet

- **Grafico de porcenataje de nulos de todas las variables**

Procederemos a calcular el porcentaje de valores nulos por columna. Esta información nos sera muy útil para tomar decisiones individuales sobre cómo abordar estos valores nulos en cada caso específico.


In [ ]:
porcentaje_nulos = (ds_original.isnull().sum() / len(ds_original)) * 100
porcentaje_nulos = porcentaje_nulos.sort_values(ascending=False)
plt.figure(figsize=(25, 8))
ax = porcentaje_nulos.plot(kind='bar', color='skyblue')

porcentaje_nulos.plot(kind='bar', color='skyblue')#,autopct=lambda p: '{:.2f}% ({:.0f})'.format(p,(p/100)*valores.sum()))
plt.title('Porcentaje de valores nulos por columna')
plt.xlabel('Columnas')
plt.ylabel('Porcentaje de Nulos')
plt.xticks(rotation=75)
plt.ylim(0, 100)
plt.gca().yaxis.set_major_formatter(PercentFormatter())
for p in ax.patches:
    ax.annotate(f'{p.get_height():.2f}%', (p.get_x() + p.get_width() / 2., p.get_height()), ha='center', va='bottom')

plt.show()

Primero decidimos en la columna agent reemplazar los valores Nan con ceros.
Luego, tomamos la decision de unir las columnas company y agent ya que describen el ID y la columna company tenia un alto porcentaje de valores NULL. En el caso que un registro no tuviera NULL en las dos columnas, decidimos dejar el valor de company.

Reemplazamos los valores NULL en la columna country con 'Undefined'
para ..

In [ ]:
ds_copia = ds_original.copy()
###agregar una C a los que sean company
ds_unificado = pd.DataFrame(ds_copia)

ds_unificado.agent.fillna(0, inplace=True)

ds_unificado['agent_company'] = ds_unificado['company'].fillna(ds_unificado['agent'])
ds_unificado = ds_unificado.drop(['company', 'agent'], axis=1)

ds_unificado.isna().any()

Mostramos la cantidad de valores NaN previo a la imputacion de la columna country

In [ ]:
ds_filtrado_agent = ds_base['country']

numero_de_NaN_agent = ds_filtrado_agent.isna().sum()
numero_de_lineas_agent = ds_filtrado_agent.shape[0]

porcentaje_de_NaN_agent = (numero_de_NaN_agent/numero_de_lineas_agent ) * 100
porcentaje_de_NaN_redondeado_agent = round(porcentaje_de_NaN_agent, 2)

valores_agent = {'numero_de_NaN': [numero_de_NaN_agent],
           'numero_de_lineas': [numero_de_lineas_agent],
           'porcentaje_de_NaN': [porcentaje_de_NaN_redondeado_agent]}


df_agent = pd.DataFrame.from_dict(valores_agent)
df_agent.head()
#Se perdieron como 20000 lineas ¿?

Asi quedaria la distribucion de country luego de la imputacion

In [ ]:
ds_filtrado_agent = ds_unificado['country']

numero_de_NaN_agent = ds_filtrado_agent.isna().sum()
numero_de_lineas_agent = ds_filtrado_agent.shape[0]

porcentaje_de_NaN_agent = (numero_de_NaN_agent/numero_de_lineas_agent ) * 100
porcentaje_de_NaN_redondeado_agent = round(porcentaje_de_NaN_agent, 2)

valores_agent = {'numero_de_NaN': [numero_de_NaN_agent],
           'numero_de_lineas': [numero_de_lineas_agent],
           'porcentaje_de_NaN': [porcentaje_de_NaN_redondeado_agent]}


df_agent = pd.DataFrame.from_dict(valores_agent)
print(df_agent)

ds_unificado.country.fillna('Undefined', inplace=True)

top_10_paises = ds_unificado['country'].value_counts().head(31)
paises = top_10_paises.plot.bar(alpha=1, title="Top 10 países con más reservas", xlabel='Cantidad de reservas', ylabel='Países')


In [ ]:
print("Hay valores NULL? :" , ds_unificado.country.isna().any())
porcentaje_nulos = (ds_unificado.isnull().sum() / len(ds_original)) * 100
porcentaje_nulos = porcentaje_nulos.sort_values(ascending=False)
plt.figure(figsize=(25, 8))
ax = porcentaje_nulos.plot(kind='bar', color='skyblue')

porcentaje_nulos.plot(kind='bar', color='skyblue')#,autopct=lambda p: '{:.2f}% ({:.0f})'.format(p,(p/100)*valores.sum()))
plt.title('Porcentaje de valores nulos por columna')
plt.xlabel('Columnas')
plt.ylabel('Porcentaje de Nulos')
plt.xticks(rotation=75)
plt.ylim(0, 100)
plt.gca().yaxis.set_major_formatter(PercentFormatter())
for p in ax.patches:
    ax.annotate(f'{p.get_height():.2f}%', (p.get_x() + p.get_width() / 2., p.get_height()), ha='center', va='bottom')

plt.show()